# ACSE-2 Coursework for Wave Propagation, November 2019


----

**This example filled-in worksheet for the coursework mentions various mistakes & omissions that I saw in the submissions, and has suggestions about various ways to do some of the coding for the tasks.**

**It also has some extra thoughts about how such methods might compare under different conditions (e.g. potential pitfalls when scaling up, etc.)**

**–Hope it's useful & informative!**

**Adrian**

----

This Jupyter notebook contains several tasks which require you to write code, as well as some explanations, into some of the coding cells.  

–Be sure to read the tasks carefully!

It also contains several coding cells that will plot the contents of various arrays for you, as well snapshots and animations of the final results. You can use these to check whether your coding works as you expect.

Note that it's well worth adding comments to your coding, so that, even if the code itself doesn't quite behave as you want, you may still be able to gain some marks by showing your intentions.

## Import libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML
import sys

----

## Create domain model properties (i.e. speed of waves), and discretisation grid

> ## –––– Task 1 ––––
>
> **– Create a model array named `c` for a domain of size 5km by 4km, with grid-spacing 10m.**
>
> **– Fill it with a wave-propagation speed that is 1.52km/s at the top-left corner, with a gradient such that the propagation speed decreases linearly in the direction heading along the diagonal, and reaching 820m/s at the bottom-right corner.**
>
> (10 marks)

In [ ]:
# REPLACE THE QUESTION-MARKS BELOW to fulfil the first part of task 1...

#  (NOTE: I've kept original lines, with question-marks, as comments)

# domain dimensions
length = 5000.0  # length = ??
depth = 4000.0   # depth = ??

# grid-spacing
dx = 10.0  # dx = ??

# now find the number of grid-points along each dimension
# ***NOTE: I should've specified that 5km is X and 4km is Z, and I think that'll be how
# most understand the description. But I wasn't precise enough, so never mind if transposed...
nx = int(length/dx)+1  # nx = ??
nz = int(depth/dx)+1   # ny = ??  # THIS WAS A MISTAKE IN WORKSHEET -should be nz, not ny
# NOTE the +1 in both of above -we should end up with 501x401 gridpoints in the domain!


# variables that we set up below, so that if the required specs change (i.e. the 1520 & 820m/s)
# then we only need to change things in *one* place in the code...
vmax = 1520.0
vdiff = vmax - 820.0


# create the array for the domain's model properties
# (can use 'empty' here since we're filling it all below)
c = np.empty((nx,nz))  # c = ??

# could create array by filling with some initial value...
# c = np.full((nx,nz),vmax)  # ...e.g. vmax
# c = np.zeros((nx,nz))      # ...or zero (since it all gets overwritten below)
# c = np.full((nx,nz),np.nan) # ...or how about filling with NaNs, so we can easily spot errors?!  :)

# Note that filling c with 1520 is *wrong* because 1520 is an integer, so c then ends up an array of ints,
# and *that doesn't change*, even when doing floating-point calculations further down for the gradient!!
#c = np.full((nx,nz),1520)  # This ends up with *integers* in array c!



# WRITE CODE BELOW to fill model properties into array c (second part of task 1)...

# precalculate various things that remain constant during double-loop, so that the code within
# double-loop is very compact, and as fast as possible (otherwise we're in danger of this taking
# more significant time to run than it needs if using double-loop below)
f = vdiff/((nx-1)**2+(nz-1)**2)
a = f*(nx-1)  # step in speed along x direction
b = f*(nz-1)  # step in speed along z direction

# Below shows expected values at corners of domain (TL,TR,BL,BR):
print('Expected TL,TR,BL,BR values: ',vmax,vmax-a*(nx-1),vmax-b*(nz-1),vmax-a*(nx-1)-b*(nz-1))

for i in range(nx):
    for j in range(nz):
        c[i,j] = vmax - a*i - b*j  # one possible way to fill with desired gradient (very compact)
        #c[i,j] = vmax - f*(i*(nx-1)+j*(nz-1))  # another way to give the same gradient (nearly as compact)

        #c[i,j] = vmax - vdiff*(i+j)/(nx+nz-2)   # different gradient that fits description (quite compact)
        #c[i,j] = vmax - 0.5*vdiff*(i/(nx-1)+j/(nz-1))  # another possible way (almost as compact)
        # I guess it's technically possible to argue that a vertical gradient still fits it...
        #c[i,j] = vmax - vdiff*j/(nz-1)  # ...but I think it misses the sense behind the task description



# Alternative way to create gradient, avoiding double-loop by filling in row by row...
c = np.empty((nx,nz))
c[:,0] = np.arange(vmax,vmax-a*(nx-0.1),-a)  # fill the first row
# NOTE ROUNDING ERROR PROBLEM with above if using vmax-a*nx for end of arange (hence why it's nx-0.1) !!
#c[:,0] = vmax - np.arange(nx)*a  # alternative way, avoiding rounding error problem
for j in range(nz-1):  # loop through the rest of the rows
    c[:,j+1] = c[:,j] - b  # fill each row from previous

    
# demonstrating the rounding-error problem seen above...
print( ((vmax-a*(nx+1))-vmax) / -a )
print( ((vmax-a*nx)-vmax) / -a )
print( ((vmax-a*(nx-1))-vmax) / -a )
print( ((vmax-a*(nx-2))-vmax) / -a )
# Expect exactly 502,501,500 & 499 from above calculations, but get *just above* for 501 case (and 499).
# This means using np.arange(vmax,vmax-a*nx,-a) in row loop above actually gives 502 elements
# for the np.arange size (rather than 501), so it all then fails due to inconsistent dimensions.
# That's why above fill code has a*(nx-0.1) instead to 'hack' the correct length for the arange. :)



# Uncomment below to demonstrate how solution will 'blow up' when violating CFL condition...
#c[:,:] = 1000.0              # whole domain is 1km/s...
#c[200:220,150:170] = 1520.0  # ...except for 20x20 square that violates CFL
# Note: also should remove obstruction from task 8 for this demonstration!


# let's print some information about the model properties you coded above
print('Domain is %d x %d grid-cells (%.1fm x %.1fm)' % (nx,nz,length,depth))
print('Grid-spacing (dx) is %.3fm' % (dx))
print('Range of speeds is %.6fm/s to %.6fm/s' % (c.min(),c.max()))
print('Speed at top-left is %.6fm/s  Speed at bottom-right is %.6fm/s' % (c[0,0],c[-1,-1]))

# There's some ambiguity in the way the gradient could be done above, so I've added below for comparison.
# As long as the above min/max speeds are at the top-left & bottom-right, and there's a linear gradient between
# the two that's pointed kinda along the diagonal, then I don't care that much about other details.
print('Speed at top-right is %.2fm/s  Speed at bottom-left is %.2fm/s' % (c[-1,0],c[0,-1]))

print(c)  # just so I can see it contains floats rather than ints!

In [ ]:
def plot_model(c):
    plt.figure(figsize=(10,8))
    plt.imshow(c.T)  # plotting the velocity model (note transposed, to orient plot in way we expect)
    plt.colorbar()
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')
    plt.title('Velocity model (m/s)')
    plt.show()

def plot_hslice(c,zgrid):
    plt.figure(figsize=(7,6))
    plt.plot(c[:,zgrid])
    plt.ylabel('propagation speed / m/s')
    plt.xlabel('x gridpoints')
    plt.title('Horizontal slice through model at z-gridpoint=%d' % zgrid)
    plt.show()

In [ ]:
# Now let's plot the model properties across the domain, and a horizontal slice
plot_model(c)
plot_hslice(c,201)

----
## Modelling constraints
#### CFL stability condition should be satisfied – the 'Courant number'.

The dimensionless Courant number, $C$, gives a measure of how far a signal will travel between grid-points within one time-step.

This 'crossing factor' is $c$.$\delta t$/$\delta x$ for speed $c$, and we want the maximum value of this within the whole domain to satisfy some constraint that depends upon the finite-difference stencil(s) in use when modelling.

i.e. we want:$\quad \text{max}(c)$.$\delta t/\delta x <= C_{max}$, where $\ C_{max}$ depends on the discretisations being used.

We can turn this around to find the maximum time-step for our model and grid-spacing, given $C_{max}$ for our discretisations:

$$\delta t_{best} = \frac{C_{max}\ \delta x}{\text{max}(c)}$$

A simple 2nd order spatial stencil, with 2nd-order time-stepping, requires the max crossing factor to (normally) be no more than ~sqrt(1/2)

> ## –––– Task 2 ––––
>
> **– The duration for the simulation should cover at least 3 seconds, with time-steps of 4.7 milliseconds.**
> 
> **– Work out the number of time-steps to cover that time (call it `nt`), and calculate the maximum Courant number within the domain.**
>
> **– Print information about the total simulation time and the maximum Courant number.**
>
> **– Is this time-step size a good choice for this system?** (Explain your answer by filling in comments at the end of the code cell.)
>
> (10 marks)

In [ ]:
# REPLACE THE QUESTION-MARKS to fulfil the first part of task 2...

time = 3.0   # time = ??
dt = 0.0047  # dt = ??


# WRITE MORE CODE to fulfil the next two parts of task 2...

nt = int(time/dt+0.9999)  # how many steps are needed to (very very nearly) cover that time
#
# Strictly speaking, to ensure we fully cover the required 3s we should do something like
# ceiling(time/nt) above (instead of adding 0.9999).
# However, we're doing a simulation here, using numerical methods that are clearly only an
# approximation to reality, so we shouldn't mind that much if we only end up at 2.99999s rather
# than 3s, since it'll not make any significant difference to what we see at that time.
# The important thing is to ensure we're only a very small percentage of a time-step away from
# the desired target time (i.e. 3s) by the end of the simulation (i.e. we don't want to end up
# with, e.g., 30% of a time-step still remaining to see the full 3 seconds of our simulation).
#
# Note that adding 1 (instead of 0.9999) does mean we could potentially end up doing an extra
# time-step, unnecessarily, if it just so happens that time/dt works out as an integer!


time = nt*dt  # turn that back into exact time for this number of steps
Courant_max = c.max()*dt/dx

print('Time-step = %.3fms  Number of steps = %d  (Total time being modelled: %.5fs)' % (dt*1000,nt,time))
print('Max crossing factor is %.5f' % Courant_max)


# WRITE COMMENTS BELOW to explain your answer to the final part of task 2...
#
# The expected stability criterion for this discretisation is that the max. crossing factor should
# be less than 1/sqrt(2) – but THAT IS NOT THE CASE for the 4.7ms time-step given here!
#
# The max Courant number (i.e. max crossing factor) is a little more than that, 0.7144>sqrt(1/2), so
# we might expect the time-stepping could become unstable when we propagate with this time-step.
# The 'best' time-step we should safely pick would be a little less than the one here (see below). 

dtbest = np.sqrt(0.5)*dx/c.max()  # this is the max time-step we can 'safely' take for this problem
ntbest = int(time/dtbest+0.9999)
print('Expected best time-step is %.3fms, with %d steps (total time: %.5fs)' % (dtbest*1000,ntbest,ntbest*dtbest))

#
# BUT...
# ...there is only a very small number of gridpoints, at top-left of the domain, where this
# crossing factor (CFL condition) is violated.
# ...also, it is well within the absorbing region (at top-left of domain), so any instability that
# starts to arise could start to get damped enough by absorption there.
#
# NOTE: This is a dangerous game to play with the CFL condition, since the consequences for a 
# simulation are catastrophic (see the commented demo code in task 1 above, where c gets filled
# with 1km/s, apart from 20x20 square that's at 1520m/s).
# But we do actually get away with it for the case given here -we survive with a time-step that's
# actually a bit 'better' (i.e. larger) than the expected best!
# (The reason we'd prefer going for larger steps is because this is going to reduce the number of
# time steps we end up having to do for the simulation to cover the required total time, hence
# reducing computation time. The difference is fairly small here of course - just 7 steps more,
# as you can see from info printed below, which is ~1% extra.)
#

----

### Create source function (ramp up to 10Hz mono-frequency wave)

> ## –––– Task 3 ––––
>
> **– Create variables that will allow you to choose the position of the point source during the time-stepping propagation loop (that is coded further down); place it half-way across the domain, and with depth 750m from the top of the domain**
>
> **– Code a loop in the `create_source` function that creates & fills in an array to contain a sine wave with frequency `freq`, but ramping up the sine wave amplitude from zero to `maxampl` (linearly) over the first 100 steps.**
>
> **– Return that array so the subsequent code cell gets its `src` array containing your source.**  
>
> (6 marks)

In [ ]:
# CREATE THE VARIABLES for the first part of task 3...

# values from specs given in task 3:
src_pos_x = length/2.0  # half-way across
src_pos_z = 750.0       # 750m down
# turn above into integer gridpoint positions:
sx = int(src_pos_x/dx)
sz = int(src_pos_z/dx)

print(sx,sz,nx,nz)  # just so I can check they look sensible!


# function to create a source that ramps up to mono-frequency sine wave
# (frequency is 'freq', time-step length is 'dt', number of steps is 'ns')
def create_source(freq,dt,ns,maxampl):

    # WRITE CODE JUST BELOW to create and return the source array to complete the rest of task 3...

    src = np.empty(ns)  # can use 'empty' here, since array gets filled below
    wdt = np.pi*2*dt*freq  # this remains constant throughout loop below, so precalculate it
    for i in range(ns):
        src[i] = maxampl*np.sin(wdt*i)*min(0.01*i,1.0)
    return src  # returning the source array


    
# below is an alternative way that avoids using a loop, and avoids touching the whole of the
# src array after it first gets filled (only 100 entries get changed, for the ramp)...

def create_source_alternative(freq,dt,ns,maxampl):

    src = maxampl*np.sin(np.pi*2*dt*freq*np.arange(ns))  # build sine wave without any ramp
    src[:100] = np.arange(0,1,0.01)*src[:100]   # put ramp onto first 100 entries
    return src

# !WARNING! -above alternative way has an issue which could potentially cause a problem!!
# I saw a similar type of problem popping up in several submissions.
# (Hint: consider the case when ns<100...)
# -not too hard to see how to rewrite to avoid it...



In [ ]:
ns = nt  # this source keeps going to the end of time
# call to function below should create a ramped 10Hz sine wave with unit amplitude and 'ns' time steps
src = create_source(10.0,dt,ns,1.0)

In [ ]:
def plot_source(src):
    plt.figure(figsize=(10,6))
    plt.plot(src)  # plot source function
    plt.xlabel('timesteps')
    plt.ylabel('amplitude')
    plt.title('Source Function')
    plt.show()

In [ ]:
# let's check that your source wavelet looks as you expect by plotting it...
plot_source(src)

### Check that the maximum frequency in the source wavelet can propagate reliably

For a simple second-order finite-difference, the minimum wavelength of a signal that we can propagate reliably over a reasonable distance is about 10 grid-points.

> ## –––– Task 4 ––––
>
> **– Calculate and show the maximum reliable propagation frequency for this model.**
>
> **– Briefly explain, then, what the resulting amplitude spectrum plot tells you.**  
(Write it as a comment where indicated in the code cell further down that calls function `plot_spectrum`.)
>
> (4 marks)

In [ ]:
# WRITE CODE BELOW to complete the first part of task 4...

min_cells_per_wl = 10.0  # minimum of 10 cells per wavelength for reasonably accurate propagation
max_freq = c.min()/(min_cells_per_wl*dx) # calculate max frq that can be modelled without numerical dispersion
print('Maximum reliable propagation frequency is about %.1fHz' % max_freq)


#### Want to avoid causing too much dispersion by keeping maximum significant frequency within this limit...

In [ ]:
def plot_spectrum(src,dt,nt,ns):
    plt.figure(figsize=(10,6))
    plt.magnitude_spectrum(np.append(src,np.zeros(nt-ns)), Fs=1/dt)  # note padding to nt points
    plt.title('Amplitude Spectrum')
    plt.xlim(0,35)
    plt.show()

In [ ]:
# checking the amplitude spectrum of the source wavelet
plot_spectrum(src,dt,nt,ns)


# GIVE YOUR ANSWER IN COMMENTS BELOW for the final part of task 4:
#
# The frequency spectrum plot shows the expected peak around 10Hz for the source wavelet.
#
# In particular, it also shows that there are frequencies in the source (up to ~11Hz) which
# are more than the maximum reliable propagation frequency (~8.2Hz), so there could well be
# dispersion as the wavefield propagates.
# In fact, pretty much *all* of the frequencies in the src are >8.2Hz, so we might well be expecting
# some pretty serious dispersion. (For reference, take a look at the 1d worksheet examples, where we
# can see some dispersion *even when the src freq. spectrum has very little above the MRPF*!)
#
# HOWEVER... the MRPF is a function of speed, which changes across the domain, and it's only near
# bottom-right of the domain where MRPF gets less than 10Hz, as speed drops towards 820m/s, so most
# of the domain is actually fine, and can propagate our 10Hz signal without too much dispersion!
#
# (Also, since src stays on, dispersion would not be so clearly visible here - the wavefront
# will not have trailing 'tails' that become more pronounced, due to being overcome by the more
# significant contribution of the wavefront that continues behind, without dying away.
# Again, for reference see the 2nd 1d worksheet, and note how the dispersion shows up as a 'trail'
# of wiggles behind the moving 'bump', and that it increases the further the bump moves.)
#

----
## Create model for absorbing boundary layers

> ## –––– Task 5 ––––
>
> **– Write code to fill in array `a` so that it has a layer of width 70 grid-points all the way around the domain so that it has a quadratic increase towards value one, which it should reach once it hits the edges of the domain.**
>
> (10 marks)

In [ ]:
a = np.zeros((nx,nz))  # initialise absorbing layer array with zeros
# Note that we can't use np.empty above because we're only changing a 70-gp layer around the edge.
# The rest of the array doesn't get touched, and we want those to be zeros!


# WRITE CODE BELOW TO FILL ARRAY 'a' ready for quadratic-increase absorbing layer (to fulfil task 5)...

# want layer around all edges of the model to be 70 cells thick
abswid = 70
invabsw2 = 1.0/(abswid*abswid)  # useful shorthand


# left and right absorbing layers
for i in range(1,abswid+1):  # 70 grid-points in layers
    a[abswid-i,:]   = invabsw2*i*i  # quadratic increase towards left boundary
    a[i-abswid-1,:] = invabsw2*i*i  # quadratic increase towards right boundary

# top and bottom absorbing layers
for i in range(1,abswid+1):  # 70 grid-points in layer
    for j in range(nx):
        a[j,abswid-i]   = max(a[j,abswid-i],   invabsw2*i*i)  # quadratic increase towards top boundary
        a[j,i-abswid-1] = max(a[j,i-abswid-1], invabsw2*i*i)  # quadratic increase towards bottom boundary



# An alternative way to fill the array, which is more compact / much neater than above -though it's
# also only sensible in this case where the thickness is the same all the way around, i.e. not suitable
# for some of the cases I gave in the worksheets where the bottom layer was thicker than the rest.

a = np.zeros((nx,nz))  # initialise absorbing layer array with zeros

for i in range(abswid):
    v = invabsw2*(abswid-i)**2
    a[i:nx-i,i] = v
    a[i,i:nz-i] = v
    a[i:nx-i,-i-1] = v
    a[-i-1,i:nz-i] = v

#### Show the absorption model

In [ ]:
# various plotting functions that get called below so you can check your absorption model...

def plot_absorbing(a):
    plt.figure(figsize=(10,6))
    plt.title('Absorption')

    plt.imshow(a.T) # plotting the absorption as a 2d colour plot (note transposed, to orient plot in way we expect)
    plt.colorbar()
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')

    plt.show()


def plot_absorb_horiz(a,zgrid):
    plt.figure(figsize=(8,4))
    plt.title('Horizontal x-section at z-gridpoint=%d' % zgrid)

    plt.plot(a[:,zgrid])  # horizontal cross-section through absorption, to show const/linear/quadratic nature
    plt.xlabel('x gridpoints')
    plt.ylabel('absorption coefficient')

    plt.show()


def plot_absorb_vert(a,xgrid):
    plt.figure(figsize=(8,4))
    plt.title('Vertical x-section at x-gridpoint=%d' % xgrid)

    plt.plot(a[xgrid,:])  # vertical cross-section through absorption, to show const/linear/quadratic nature
    plt.xlabel('z gridpoints')
    plt.ylabel('absorption coefficient')

    plt.show()

In [ ]:
plot_absorbing(a)
plot_absorb_horiz(a,200)
plot_absorb_vert(a,200)

In [ ]:
# scales the absorbing factors in array a by the appropriate amounts at each point in space...

#absfact = 0.0  # to switch off absorbing layer
#absfact = 0.1  # without predictive boundary
absfact = 0.05  # with predictive boundary

a[:,:] = a[:,:]*c[:,:]*(dt/dx)*absfact

## Predictive boundary condition

In [ ]:
# set up crossing-factor arrays for use at edges with 'predictive' boundary condition
C_zmin = c[:,0]*dt/dx   # along top edge
C_zmax = c[:,-1]*dt/dx  # along bottom edge
C_xmin = c[0,:]*dt/dx   # along left edge
C_xmax = c[-1,:]*dt/dx  # along right edge

# for use without absorbing layers:
#predfact_zmin = 1.0
#predfact_zmax = 1.0
#predfact_xmin = 1.0
#predfact_xmax = 1.0

# for use with absorbing layers:
predfact_zmin = 0.98
predfact_zmax = 0.98
predfact_xmin = 0.98
predfact_xmax = 0.98

# to switch off predictive boundaries:
#predfact_zmin = 0.0
#predfact_zmax = 0.0
#predfact_xmin = 0.0
#predfact_zmax = 0.0


# some extra things that might be useful during task 8...
C_zmid = c[:,149]*dt/dx
predfact_zmid = 1.0


----
## Final preparations – line of detectors & time-stepping arrays

> ## –––– Task 6 ––––
>
> **– Create an array that can be used to simulate a line of detectors that goes all the way across the domain, that will be for recording the wavefield over time that crosses each grid-point in that line.**
>
> **– Create a variable that allows choice for the depth for this line of detectors during the `propagate` function (which is in a code cell later down the notebook), giving it a depth that corresponds to 3km from the top of the domain.**
>
> **– Complete code to create the three arrays to be used for time-stepping during the `propagate` function.**
>
> **– Add comments giving brief explanation what each array will be used for during the simulation.**
>
> (5 marks)

In [ ]:
# WRITE CODE BELOW FOR A LINE OF DETECTORS (for first two parts of task 6)...

r = np.zeros((nx,nt))  # an array to store receiver data every time-step (and to plot later)

detect_depth = 3000.0  # define it from specs given in task above (i.e. 3km)...
rz = int(detect_depth/dx)  # ...and turn it into an integer grid-point depth


# REPLACE QUESTION-MARKS BELOW TO CREATE THREE ARRAYS HERE for rest of task 6
# ALSO complete a brief comment for each explaining what it is for...

u_cur = np.zeros((nx,nz))  # wavefield at current step       # u_cur = ??   # this is... what??

# NOTE: I've removed this array, since I've made some optimisations to the propagation kernel
# so that it only needs to use *TWO* arrays rather than one...
#u_prv = np.zeros((nx,nz))  # old wavefield at previous step  # u_prv = ??   # this is... what??

# Due to the above-mentioned optimisations, the array below will actually contain both the
# previous AND next wavefields (it overwrites the previous with the next during propagation)
u_nxt = np.zeros((nx,nz))  # next & previous wavefield      # u_nxt = ??   # this is... what??


In [ ]:
# prepares an array to store wavefield snapshots for plotting later
snapshot_gap = 12 # set sampling rate used to store wavefield (every 12 timesteps)
wavefield = np.zeros((int(nt/snapshot_gap), nx, nz)) # array to store wavefields every 12 timesteps
print('Storing %d wavefields (every %dth out of %d)' % (wavefield.shape[0],snapshot_gap,nt))

----
# Simulation

> ## –––– Task 7 ––––
>
> **– Fill in the code for the `propagate` function below, using the three arrays you created earlier, so that it propagates the wavefield from the source through the domain during the time loop.**
>
> **– In this coding, during the calculation of the new wavefield for a time-step, include the implementation of absorbing layers, using array `a` (which you populated in task 5).**
>
> **– Also include the coding for the predictive boundaries, using the factors in arrays `C_xmin`, `C_xmax`, `C_zmin` & `C_zmax` (see the relevant coding cell earlier in this notebook).**
>
> **– Also include code to fill in the wavefield values along the line of receivers that you set up in task 6.**  
>
> (22 marks)

> ## –––– Task 8 ––––
>
> **– Think of a way to simulate an 'obstruction' across the whole width of the domain at depth 1.5km, such that the wavefield cannot cross it, meaning the wavefield is constrained within the top ~1.5km of the domain. Add code into the time-stepping loop to implement that.**  
(Note that it's ok for the wavefield to 'bounce' off this obstruction -you don't need to try to add an absorbing layer to make the wavefield fade away before it reaches the obstruction.)
>
> **– But leave two 'holes' in this obstruction so that the wave can pass through them. Make the holes each have width 7 grid-points, and position the two holes so they are either side of a vertical line down the middle of the domain, and such that there are 71 grid-points between the holes (i.e. so there is still 'obstruction' for 71 grid-points along the line between the two holes).**
>
> **– After checking above all works as you would expect, include more code to implement a predictive boundary layer just above the obstruction, in order to reduce the reflection of the wavefield that 'bounces' off the obstruction.**  
(You can make use of `C_zmid` and `predfact_zmid` that were defined in the relevant coding cell earlier in this notebook.)
>
> (18 marks)

In [ ]:
# a useful variable – shorthand for something that appears in expressions below
dtdx2 = (dt*dt)/(dx*dx)


obs_depth = 1500.0  # depth of obstruction
od = int(obs_depth/dx)  # turn it into grid-point depth

# set parameters from specs given in task above 
hole_wid = 7
hole_gap = 71

mid = int(length/(2*dx))  # middle gridpoint of the domain (250)

# work out positions from given specs, and put them into vars so we don't need to
# change lots of things in the propagation code when the specs change, or when we
# add more related things (e.g. can use these vars for the obstruction AND for the
# implementation of the predictive boundary above the obstruction)
l2 = int(mid - (hole_gap-1)/2)  # right gridpoint after left hole (i.e. start of 71pts of obstruction)
r1 = int(mid + (hole_gap+1)/2)  # left gridpoint of right hole (i.e. start of 7pts of left hole)
l1 = l2 - hole_wid  # left gridpoint of left hole (i.e. start of 7pts of left hole)
r2 = r1 + hole_wid  # first gridpoint beyond right hole (i.e. *after* right of right hole)

# just to check various things are as we'd expect...
print(depth,od,od*dx,obs_depth)
print(length,dx,mid*dx)
print(mid,l1,l2,r1,r2)  # should end up as 250, 208, 215, 286, 293
# Note how easy it can be to get the positions wrong here, esp. due to the way python ranges work...!
# (That's why l2 above has hole_gap-1, while r1 has hole_gap+1, for example, because one is the start
# of a range, while the other is the end of a range which is NOT included for a python range!)


# One way to create obstruction is to zero the speed in the model, which prevents propagation through
# the zeroed gridpoints:
c[:l1,od] = 0.0
c[l2:r1,od] = 0.0
c[r2:,od] = 0.0
#
# just to show model after adding above obstruction
plot_model(c)
#
# Note that above has a potential drawback in that we've now LOST whatever model properties (i.e. speeds)
# were in those positions. For the simple case developed in this worksheet it doesn't matter, since we
# never use them anywhere else. But it could've become an issue if I'd asked for some other propagation
# with the original gradient (and without the same obstructions, or with different obstructions).


# The other simple way to make the obstruction is to zero the wavefield at each step (e.g. to set u_nxt
# to zero right after updating it, or to set u_cur to zero right after cycling the wavefields at the end
# of a time-step, or to zero u_cur at the start of a time step) -see relevant code in time loop below...
#
# One potential drawback of that method, compared to the previous method (zeroing c), is that it has to
# be done at every time-step, which means a bit of extra computation.
# Doesn't make any significant difference here, since everything is so small, but worth bearing in mind!


def propagate(u1,u2):

    # ensure wavefields start off zero
    u1[:,:] = 0.0
    u2[:,:] = 0.0

    # begin time-stepping loop...

    for i in range(nt):


        # WRITE CODE BELOW TO COMPLETE TASKS 7 and 8...

        # find new wavefield, u1, throughout domain (apart from edges)
        u1[1:-1,1:-1] = ( (2-a[1:-1,1:-1]**2)*u2[1:-1,1:-1] - u1[1:-1,1:-1]*(1-a[1:-1,1:-1]) \
                           + (c[1:-1,1:-1]**2) * dtdx2 * \
                                (-4*u2[1:-1,1:-1]+u2[:-2,1:-1]+u2[2:,1:-1]+u2[1:-1,:-2]+u2[1:-1,2:]) ) \
                         / (1+a[1:-1,1:-1])
        # NOTE that I've only used TWO arrays (rather than the three I used in the worksheets).
        # I've 'merged' what was u_nxt & u_prv into a single array (got rid of u_prv), since above
        # writes straight into one of the arrays it's also referencing from (u1) without having
        # to worry about losing things that might be needed.
        # It's easier to see this can still work by looking at it when it was coded as a double-loop
        # in the first 2d worksheet...
        #for ix in range(1,nx-1):
        #    for iz in range(1,nz-1):
        #        u_nxt[ix,iz] = 2*u[ix,iz] - u_prv[ix,iz] + (c[ix,iz]**2) * dtdx2 *  \
        #                            (-4*u[ix,iz]+u[ix-1,iz]+u[ix+1,iz]+u[ix,iz-1]+u[ix,iz+1])
        # Note how it only references u_prv[ix,iz] when setting u_nxt[ix,iz]...
        # That means we can throw away u_prv and just use u_nxt, and write it as:
        #        u_nxt[ix,iz] = 2*u[ix,iz] - u_nxt[ix,iz] + (c[ix,iz]**2) * dtdx2 *  \
        #                            (-4*u[ix,iz]+u[ix-1,iz]+u[ix+1,iz]+u[ix,iz-1]+u[ix,iz+1])
        # ...and then we can also 'cycle' the wavefields by just swapping the *references* for
        # u_nxt & u_cur, instead of swapping the values between them (see towards bottom of cell).
        #
        # Of course, this does slightly 'stretch' the naming of the arrays (since u_nxt would
        # be the previous wavefield at the start of a step, and only becomes the next wavefield
        # in the middle of the step as it's being updated, before it gets 'cycled'). That's kinda
        # why I ended up going with the rather less descriptive names u1 & u2 instead...
        #

        
        # obstruct wavefield propagation, except through two holes (i.e. double-slit)
        u1[:l1,od] = 0.0
        u1[l2:r1,od] = 0.0
        u1[r2:,od] = 0.0
        # Above is the other way to create an obstruction, by zeroing the wavefield at each step.
        #
        # Note that we can get away with zeroing only a single layer for this example, since the
        # discretisation used for propagation is the very simple 2nd-order in space.
        # This a gridpoint only references its nearest neighbours when getting updated.
        # If we were to use 4th-order in space, the update 'sees' two gridpoints away, so we
        # would need to zero across *two* layers to prevent the wavefield from 'leaking'
        # through the obstruction (i.e. need to zero across both od and od+1).


        # implement the predictive boundary just above obstruction, to reduce reflection from it
        # (Note gaps that are aligned above the holes in the obstruction!)
        u1[:l1,od-1] = (u2[:l1,od-1]*(1.0-C_zmid[:l1]) + u2[:l1,od-2]*C_zmid[:l1])*predfact_zmid
        u1[l2:r1,od-1] = (u2[l2:r1,od-1]*(1.0-C_zmid[l2:r1]) + u2[l2:r1,od-2]*C_zmid[l2:r1])*predfact_zmid
        u1[r2:,od-1] = (u2[r2:,od-1]*(1.0-C_zmid[r2:]) + u2[r2:,od-2]*C_zmid[r2:])*predfact_zmid
        # Note that I've set values at od-1 (which is above obstruction), but also used values
        # from layer od-2 (since that's the direction from which the waves are propagating to
        # the obstruction, i.e. the direction away from the obstruction).


        # implement predictive boundary condition for top and bottom of domain
        u1[:,0] = (u2[:,0]*(1.0-C_zmin[:]) + u2[:,1]*C_zmin[:])*predfact_zmin
        u1[:,-1] = (u2[:,-1]*(1.0-C_zmax[:]) + u2[:,-2]*C_zmax[:])*predfact_zmax

        # implement predictive boundary condition for left and right of domain
        u1[0,:] = (u2[0,:]*(1.0-C_xmin[:]) + u2[1,:]*C_xmin[:])*predfact_xmin
        u1[-1,:] = (u2[-1,:]*(1.0-C_xmax[:]) + u2[-2,:]*C_xmax[:])*predfact_xmax


        # inject the source into the domain (doesn't matter for this case whether it's set or added)
        if i<ns:
            u1[sx,sz] = src[i]
        # NOTE that I made a mistake in the lecture worksheets...
        # The second 1d worksheet had src[i+1] instead of src[i] (along with "if i+1<ns...")
        # This was due to a 'historical accident' in the way I first started injecting the src
        # –I was setting an initial shape src in the wavefield (u_cur) before starting the time-loop
        # (similar to the very first worksheet on normal modes), so it ended up having src[0] injected
        # into it before the time loop started.
        # It's not appropriate for the way I finally ended up coding it all, but I never noticed,
        # so never got around to fixing it...  :(
        # In general it just means that the propagation is ahead by one step throughout.
        # But in the case of this worksheet it also means that no src gets injected on the final
        # step (because that's the only step where NOT i+1<ns). The src here was meant to continue
        # even to the final step (since ns=nt) -though it makes no practical difference to any plots
        # or animations, since the mistake has no chance to propagate beyond a single gridpoint.


        # fill in values at receivers
        r[:,i] = u1[:,rz]

        if (i+1)%snapshot_gap == 0: # store the current wavefield u on every 12th step
            wavefield[int((i+1)/snapshot_gap-1)] = u1[:,:]

        # cycle wavefields ready for next time-step (cannot do it quite like this here, due to
        # optimisation of using only two wavefields instead of three!)
        #u_prv[:,:] = u_cur[:,:]
        #u_cur[:,:] = u_nxt[:,:]

        # Below is a much more efficient way to cycle - it avoids copying anything by just rotating
        # the array *references*!  :)
        ut = u1  # temporary reference to remember what u1 referenced
        u1 = u2  # change u1 reference to now 'point' to u2
        u2 = ut  # change u2 so it 'points' to what u1 referenced before we overwrote it

        if (i+1)%10==0:  # show progress every 10 steps
            sys.stdout.write('Done step %d (of %d)\r' % (i+1,nt))


    print('Finished %d time-steps' % (nt))

In [ ]:
propagate(u_cur,u_nxt)

## Plot wavefield at different times

In [ ]:
def plot_snapshot(plot_time,bounds):
    plt.figure(figsize=(10,7))
    plt.imshow(wavefield[int(plot_time/(dt*snapshot_gap))].T,   # note the wavefield was transposed
               vmin=-bounds, vmax=bounds, cmap='RdBu', interpolation='bilinear')
    plt.title('Wavefield at about %.2fs' % (plot_time))
    plt.colorbar()
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')
    plt.show()

In [ ]:
plot_snapshot(1.0,0.06)
plot_snapshot(2.0,0.06)


## Plot data at receivers

In [ ]:
def plot_at_receivers(r,bounds):
    plt.figure(figsize=(10,7))
    plt.imshow(r.T, cmap='RdBu', interpolation='bilinear', aspect='auto', 
               vmin=-bounds, vmax=bounds,   # set bounds for colourmap data
               extent=(0,nx,time,0))  # set bounds for axes
    plt.title('Receiver Data')
    plt.colorbar()
    plt.xlabel('Receiver number')
    plt.ylabel('Time / s')
    plt.show()

In [ ]:
plot_at_receivers(r,0.04)

In [ ]:
# NOTE: I've modified this a bit to plot values from *two* arrays, just to demonstrate
# that they end up exactly the same (i.e. you only see a single orange curve, which is
# from d2, since it lies right on top of the blue one, which comes from d1)...

def plot_rms(d1,d2):
    plt.figure(figsize=(10,6))
    plt.plot(d1)

    plt.plot(d2)

    plt.xlabel('receiver number')
    plt.ylabel('amplitude')
    plt.title('RMS at receivers')
    plt.show()

> ## –––– Task 9 ––––
>
> **– Write code to calculate the RMS (root-mean-squared) value over time at each receiver grid-point and put it into an array.**
>
>**– Call the `plot_rms` function to plot the RMS values.**
>
> **The RMS value for a receiver corresponds to the overall intensity of the wavefield arriving at that receiver.  
> – Describe the pattern you see shown in the plot below.  
> – Does it remind you of a well-known optics experiment/demonstration?  
> – But what do you think is slightly different about the intensity plot shown here, and why?**
>
> (15 marks)

In [ ]:
# WRITE CODE BELOW to find RMS at receivers for the first two parts of task 9...

rms1 = np.empty(nx)  # create array to hold the rms values
rms2 = np.empty(nx)  # create array to hold the rms values


for i in range(nx):  # loop over receivers
    s = 0.0
    for j in range(nt):  # loop over time-steps...
        s = s + r[i,j]*r[i,j]  # ...summing squares
    rms1[i] = np.sqrt(s/nt)  # find root-mean for resulting sum of squares

    # Single-line alternative ways using numpy methods (i.e. no explicit j-loop):
    #rms2[i] = np.sqrt(np.mean(np.square(r[i,:])))
    #rms2[i] = np.sqrt(np.mean(r[i,:]**2))
    rms2[i] = np.linalg.norm(r[i,:])/np.sqrt(nt)



plot_rms(rms1,rms2)  # use the function to plot the intensity


# Below does the whole RMS calculation in a single line!
rms = np.sqrt(np.mean(np.square(r),axis=1))

plot_rms(rms1,rms)  # just to show it ends up exactly the same as above

# However, worth pointing out that above could lead to a potential issue for much larger domains
# if there are lots of receiver points, and lots of time-steps.
# In that case the 2d 'r' array is rather large (e.g. consider 100000 receivers x 10000 steps...)
# The np.square(r) above may then create a temporary 2d array the same size as r (each element
# being the square of its corresponding element in r) – so an extra array of 1000000000 elements!
# In the case of this worksheet, everything is small enough that it doesn't matter – but it's
# something to watch out for, i.e. beware just how it easy it can potentially be, when using such
# methods conveniently provided by numpy, to 'suck' a whole huge chunk of memory away without even
# realising (until your machine grinds to a halt!)



# WRITE COMMENTS BELOW to answer the final part of task 9...
#
# The plot oscillates between high and low intensity, with highest peaks & lowest troughs
# near the middle.
# The value falls off to zero towards each end.
#
# It's very much like the interference pattern seen in the 'double-slit' experiment, where
# monochromatic light from a laser is shone though a pair of thin slits close together.
#
# But it's not quite symmetric – the peaks/troughs on the left are slightly wider than the
# corresponding ones on right, because the wave-propagation speed changes across the domain.
# (Means the troughs & peaks don't lie along straight lines in space, but have a slight curve,
# as can be seen by looking carefully at the final frame of the animation below.)
#
# Furthermore, unlike the double-slit experiment, the troughs don't reach close to zero here.
# This is because the detector 'screen' is relatively close to the slits - it's 'near-field'.
# (In other words, the distance between slits and screen is comparable to the separation
# between the slits -whereas the standard optical two-slit experiment is not 'near-field',
# i.e. the distance to the screen is much much greater than distance between slits).
# Also, the distance between slits (compared to wavelength of waves) and size of the slits
# have an effect on the details of the interference pattern. (I think to perfectly cancel,
# the wavelength should be significantly larger than the slits? That's not usually the case
# in an optical double-slit experiment, but the troughs & peaks for light hitting the screen
# still end up giving high enough contrast that it's not obvious.)
#

## Make a movie! 

(Partly for fun – but also so that, as you develop the code during tasks 8, 9 & 10 above, you can visualise what is happening to the wavefield across time, which should help you to check that your code is behaving as you would expect...)

In [ ]:
def create_animation(bounds):
    fig = plt.figure(figsize=(10,8))

    plt.title('Wavefield')
    plt.xlabel('x gridpoints')
    plt.ylabel('z gridpoints')

    n = wavefield.shape[0]
    imgs = []
    for i in range(n):
        if i%20==0:  # show progress every 20 frames
            sys.stdout.write('Done %d of %d\r' % (i+1,n))
        img = plt.imshow(wavefield[i].T, vmin=-bounds, vmax=bounds, cmap='RdBu',
                         animated=True, interpolation='bilinear')
        imgs.append([img])

    print('Finished plots for frames, building animation...')

    ani = anim.ArtistAnimation(fig, imgs, interval=50, blit=True)

    plt.close(fig)  # prevent final frame plot from showing up inline below

    return ani

In [ ]:
ani = create_animation(0.05)
print('Preparing HTML (takes a little while...)')
HTML(ani.to_jshtml())